# Bias Invariant RNA-Seq Annotation 

Welcome to this notebook where we'll run an example using our novel RNA-Seq annotation method.

In this notebook you will be able to reproduce some of our results yourself!

We'll go through the following steps:

<ol>
<li>Install the source code for the DA model as published on our github repo</li>
<li>Load training, test and bias injection data sets</li>
<li>Run and evaluate a full training cycle for our model</li>
</ol>





Befor we can import we first have to install the package, run the following cell.<br>

In [2]:
!pip install .

In [3]:
from src import da_model, load_data

Next we'll load some source, target and bias data. We'll reproduce the results for the DA G+S-T expereiment for tissue prediction as described in our paper. <br>
As source data we load all the GTEx data originally used in the paper as well as all the SRA data as bias data. As target we'll load a random subset (frac=0.5) of the origianl TCGA test data. Using a subset of the TCGA data saved some space and time but will lead to comparable results.

In [4]:
source, target, bias = load_data.load_data()

BIRA comes with a number of hyperparameters that can be chosen freedly, here we provide the parameters chosen in the paper for this experiment.

<ul>
    <li>source_layers: a list of integers representing the number of nodes to be used per layer for the source and bias mapper, [512] will create one layer with 512 nodes</li>
    <li>classifier_layers: a list of integers representing the number of nodes to be used per layer for the classifier layer, [] will only create a single output layer with n=classes</li>
    <li>lr: learning rate applied in the second training cycle
    <li>classes: number of classes in the data</li>
    <li>batch_size: batch size</li>
    <li>margin: size of margin applied in triplet loss
    <li>print: True / False, if test accuracy should be printed after every epoch during the second training cycle    
</ul>

In [8]:
config = {'mapper_layers': [512],
          'classifier_layers': [],
          'lr': 0.0005,
          'classes': 16,
          'batch_size': 64,
          'margin': 5,
          'print': True}

Finally we start with the first training cyle, here we train the source mapper and the classification layer as a vanilla MLP

In [9]:
model = da_model.DaModel(source, target, bias, config=config)
model.train_source_mapper(epochs=10)
model.eval_source_mapper()

The accuracy above is what we achieved using GTEx to train a MLP to predict TCGA.
Note that we use a different network than the MLP G-T we present in the paper, so results my vary.
Let's see if we can do better by injecting some SRA data set biases:

In [11]:
model.train_bias_mapper(epochs=10)